In [26]:
import numpy as np
import math

import xarray as xr


import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

g = 9.8

In [27]:
Years = ["2000"]

Months = []
for imon in range(1,13): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(1,30,3):
    Days.append(str(iday).zfill(2))
    
print(Years)
print(Months)
print(Days)

['2000']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['01', '04', '07', '10', '13', '16', '19', '22', '25', '28']


In [28]:
# Months = ['12']
# Days = ['28']

In [29]:
# directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um/'+Year+'/'
directory = '/n/home12/hongwei/Hongwei_holyscratch01/Simulation_0.2um/'+Years[0]+'/'


N_head = 5 # first 5 lines are head lines, not include data
N_column = 4 # time, lon, lat, lev

# Nx = 36
# Ny = 21
# Nz = 5
# N_parcel = 3780

Nx = 24
Ny = 21
Nz = 6
N_parcel = Nx*Ny*Nz

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
    
# filename = "traj_"+Year+Months[0]+Days[0]+"_trace.1"
# file1 = open(directory+filename, 'r')
# Lines = file1.readlines()
# print(Nt,"days")


# for DT/DZ compare:
# Criterion = -2.0

In [38]:
# loop for all traj files


for year in Years:
    directory = '/n/home12/hongwei/Hongwei_holyscratch01/Simulation_0.2um/'+year+'/'
    for month in Months:
        for day in tqdm(Days):
            
            
            # (1) read original data from traj files
            filename = "traj_"+year+month+day+".1"
            file1 = open(directory+filename, 'r')
            Lines = file1.readlines()

            Nt = int( (len(Lines)-4)/N_parcel - 1 )
                
            data = np.zeros((N_parcel, Nt, N_column))

            count = 0
            # Strips the newline character
            for line in Lines:
                count += 1
    
                if count>=5:
                    i = count-5
                    i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
                    i_t = i%(Nt+1)
        
                    if i_t!=0:
                        a = line.split()

                        if i_t==1:
                            if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                        data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                        data[i_parcel,i_t-1,1] = float(a[1]) # lon
                        data[i_parcel,i_t-1,2] = float(a[2]) # lat
                        data[i_parcel,i_t-1,3] = float(a[3]) # lev
        
            file1.close()
            
            
            # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
            data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

            for ix in range(Nx):
                for iy in range(Ny):
                    for iz in range(Nz):
                        i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                        data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
            
            
            # (3) save mean lat
            with open('./Mean_lat/Mean_lat_'+year+month+day+'.txt', 'w') as f:
                for it in range(Nt):
                    
                    # abs(lat)
                    f.write(  str( data2[0,0,0,it,0] ) + ',' \
                            + str( np.mean( abs(data2[0,:,:,it,2]) ) ) + ',' \
                            + str( np.mean( abs(data2[1,:,:,it,2]) ) ) + ',' \
                            + str( np.mean( abs(data2[2,:,:,it,2]) ) ) + ',' \
                            + str( np.mean( abs(data2[3,:,:,it,2]) ) ) + ',' \
                            + str( np.mean( abs(data2[4,:,:,it,2]) ) ) + ',' \
                            + str( np.mean( abs(data2[5,:,:,it,2]) ) )  )
                            
                    f.write('\n')
            f.close()

            
            # (4) save mean lev
            with open('./Mean_lev/Mean_lev_'+year+month+day+'.txt', 'w') as f:
                for it in range(Nt):
                    
                    # abs(lev)
                    f.write(  str( data2[0,0,0,it,0] ) + ',' \
                            + str( np.mean( data2[0,:,:,it,3] ) ) + ',' \
                            + str( np.mean( data2[1,:,:,it,3] ) ) + ',' \
                            + str( np.mean( data2[2,:,:,it,3] ) ) + ',' \
                            + str( np.mean( data2[3,:,:,it,3] ) ) + ',' \
                            + str( np.mean( data2[4,:,:,it,3] ) ) + ',' \
                            + str( np.mean( data2[5,:,:,it,3] ) )  )
                            
                    f.write('\n')
            f.close()

 50%|█████     | 5/10 [03:28<03:28, 41.78s/it]


KeyboardInterrupt: 

In [ ]:
print('./Lifetime/Lifetime_'+year+month+day+'.txt')